In [1]:
# This notebook implements the Wessels, 2016 method for classifying LUCAS in t1 using reference samples from t0.
# We use the CCDC binary change no change mask to mask out areas of change between t0 and t1. The mask is based on Landsat-8 imagery..
# We use the 2015 land cover map from pflugmacher et al., 2019 when sampling reference data for t1.

In [1]:
import ee

ee.Authenticate(auth_mode='notebook')
ee.Initialize(project = 'ee-gsingh')

from geeml.utils import eeprint

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [2]:
# Import other packages used
%matplotlib inline
import pandas as pd
import geopandas as gpd
from shapely import wkt
import geemap
import numpy as np
import random, time
import matplotlib.pyplot as plt
from scipy.stats import norm, chi2

from pprint import pprint 

c:\ProgramData\Anaconda3\envs\erthy\Lib\site-packages\geemap\conversion.py:23: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
# Import 2015 land cover map from pflugmacher et al., 2019
lc = ee.Image('projects/ee-gsingh/assets/postdoc/pflaugmacheretal2019_landcover_2015').select('b1').rename('landcover')
lc.bandNames()
# Reclassify to match LUCAS classes
def reclassify_landcover(landcover_image):
    """
    Reclassify landcover image to match target classes:
    {'Artificial land': 0, 'Bare land': 1, 'Cropland': 2, 'Grassland': 3, 
     'Shrubland': 4, 'Water': 5, 'Wetland': 6, 'Woodland': 7}
    """
    
    # Create the reclassification mapping
    original_values = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    new_values = [
        -1,  # 0: unclassified - using -1 to mask later
        0,   # 1: artificial land -> Artificial land
        2,   # 2: cropland seasonal -> Cropland
        2,   # 3: cropland perennial -> Cropland
        7,   # 4: forest broadleaved -> Woodland
        7,   # 5: forest coniferous -> Woodland
        7,   # 6: forest mixed -> Woodland
        4,   # 7: shrubland -> Shrubland
        3,   # 8: grassland -> Grassland
        1,   # 9: bare land -> Bare land
        5,   # 10: water -> Water
        6,   # 11: wetland -> Wetland
        5    # 12: snow ice -> Water
    ]
    
    # Using ee.Image.remap()
    reclassified = landcover_image.remap(original_values, new_values)
    
    # Mask out unclassified pixels (value -1)
    reclassified = reclassified.updateMask(reclassified.neq(-1))
    
    return reclassified

lc_reclassified = reclassify_landcover(lc)
lc_reclassified.bandNames()

In [4]:
aoi = lc_reclassified.geometry().bounds()


In [11]:
Map = geemap.Map()
Map.addLayer(aoi, {}, 'Area of Interest')
Map.centerObject(aoi, 10)
Map

Map(center=[52.83223389470395, 9.76421064991148], controls=(WidgetControl(options=['position', 'transparent_bg…

In [12]:
# binary chnage no change mask
# ---- INPUT -------------------------------------------------
ccdResults = ee.ImageCollection("GOOGLE/GLOBAL_CCDC/V1") 
geom = aoi

# ---- PROCESS ------------------------------------------------
img = ee.Image(ccdResults.filterBounds(geom).mosaic())
change      = img.select('tBreak')
change_prob = img.select('changeProb')

# Set the time range we want to use and get as mask of 
# places that meet the condition.
mask = (change.gt(2015)          # > start year
        .And(change.lte(2018))   # ≤ end year
        .And(change_prob.eq(1)))   # prob > 0.5

# Obtain the number of breaks for the time range.
num_breaks = mask.arrayReduce(ee.Reducer.sum(), [0]).neq(0)

# ---- VISUALISE (optional) ----------------------------------
Map = geemap.Map()
Map.centerObject(geom, 9)
Map.addLayer(change_prob, {}, 'change prob', shown=False)
Map.addLayer(num_breaks.eq(0), {'min':0,'max':1}, 'zero‑break pixels')
Map

Map(center=[52.83223389470395, 9.76421064991148], controls=(WidgetControl(options=['position', 'transparent_bg…

In [16]:
# Proportion of KZN area samppled in Wessels et al., 2016 (n samples*area of pixel in sqkm/area of study area in sqkm)
prop = ((1100000*900)/1000000)/92100
area = aoi.area(5).divide(1e6).divide(2).getInfo()  # Area of aoi in km2
print('Area of the area of interest in km2: %s'%area)
print('Proportion of KZN area sampled in Wessels: %s'%prop)
# number of required samples
nSamples = (area*prop)*1000000/100
print('Number of required samples: %s'%nSamples)
# number of min classes (at least 1% of sample size)
print('Number of minimum samples per class: %s'%int(nSamples*0.01+1))
# Colditz, R.R., 2015. An evaluation of different training sample allocation schemes for discrete and continuous land cover classification using decision tree-based algorithms. Remote Sensing, 7(8), pp.9655-9681.

Area of the area of interest in km2: 12111422.501415035
Proportion of KZN area sampled in Wessels: 0.010749185667752443
Number of required samples: 1301879291.6830492
Number of minimum samples per class: 13018793


### Extracting data to work locally with scikit-learn - reproducing wesssels et al., 2016

#### Random forest

In [6]:
# Steps to follow:
# import s2 20018 classification from case study 1
# Apply change mask
# Calculate area proportions for each class
# Calculate number of samples per class
# perform stratified sample of 2023 composite with 2018 classified image
# Train 2023 model, evaluate.

In [14]:
import ee
try:
    ee.Authenticate(auth_mode='notebook')
    ee.Initialize(project = 'ee-gsingh')
except: 
    ee.Authenticate()
    ee.Initialize()

import geemap
from geeml.utils import eeprint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#### Extracting 2018 data from Earth Engine

### Sample 2015 classification, and 2018 Landsat composite

In [29]:
test = lc_reclassified.updateMask(num_breaks.arrayGet([0])).rename('class').stratifiedSample(
    numPoints=10000,  # must be 0 if you're using classValues & classPoints
    classBand='class',
    region=aoi,
    scale=30,
    geometries=True,  # set to True if you want point geometry
    seed=42
)

In [30]:
# Export the FeatureCollection to an Earth Engine asset
def export_fc_to_asset(fc, asset_id, description=None):
    if description is None:
        description = f'export_{asset_id.replace("/", "_")}'
    
    task = ee.batch.Export.table.toAsset(
        collection=fc,
        description=description,
        assetId=asset_id
    )
    task.start()
    return task

# Example usage for your stratified sample:
export_stratified_task = export_fc_to_asset(
    test,  # your FeatureCollection
    'projects/ee-gsingh/assets/postdoc/wessels_stratified_10ksample_2018',
    'Export stratified sample to asset'
)

print(f"Export task started: {export_stratified_task.id}")


Export task started: NGBW6AEMLCBDB5FCXDEMTOWD


In [11]:
def cloudMaskL8(image):
    # Landsat 8 surface reflectance data comes with a 
    # QA_PIXEL band (CFMask) to mask unwanted pixels.

    #  Bit 0 - Fill
    #  Bit 1 - Dilated Cloud
    #  Bit 2 - Cirrus
    #  Bit 3 - Cloud
    #  Bit 4 - Cloud Shadow
    clouds = image.select('QA_PIXEL').bitwiseAnd(0b1000).eq(0)
    cirrus = image.select('QA_PIXEL').bitwiseAnd(0b1100).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

    mask = clouds.Or(cirrus).Or(saturationMask)

    #   Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)

    #  Replace the original bands with the scaled ones and apply the masks.
    return image.addBands([opticalBands], None, True)\
        .updateMask(mask)

def createLandsatComposite(imageCollection: str, points: ee.Geometry, year: int):
    """Creates a composite image of a point over a given year (uses 1 month period for 2018 and 2 months for 2023).
    
    Args:
        imageCollection (str): The collection to use for the composite.
        point (ee.Geometry): The point to create the composite for.
        year (int): The year to create the composite for.
    
    Returns:
        ee.Image: The composite image.
    """

    # Get the image collection
    ic = ee.ImageCollection(imageCollection)

    startDate = f'{year}-01-01'
    endDate = f'{year+1}-01-01'

    # Filter the collection to the start and end dates, and point
    medianImage = ic.filterDate(startDate, endDate).filterBounds(points)\
    .map(lambda image: cloudMaskL8(image))\
    .median()
    
    return medianImage.select(['SR_B2','SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])

In [ ]:
from tqdm.auto import tqdm
import os
pts = ee.FeatureCollection("projects/ee-gsingh/assets/postdoc/wessels_stratified_10ksample_2018")
grid = aoi.coveringGrid(ee.Projection('EPSG:4326').scale(2.5, 1.5))
gridList = grid.toList(972)
csv_file_path = r"C:\Users\coach\myfiles\postdoc\Invasives\data\LUCAS\Wessels_L8_30m_2018.csv"

# Remove existing file to ensure clean start
if os.path.exists(csv_file_path):
    os.remove(csv_file_path)

for i in tqdm(range(0, 972)):
    fts = pts.filterBounds(ee.Feature(gridList.get(i)).geometry())

    if fts.size().getInfo() == 0:
        print(f"No features found in grid cell {i}. Skipping...")
        continue

    composite2018 = createLandsatComposite(imageCollection= ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filter(ee.Filter.lte('CLOUD_COVER', 70)),
                points=fts.geometry(),
                year=2018)
    data = composite2018.sampleRegions(collection = fts, scale = 30, geometries = True)

    # Conversion to GeoDataFrame
    gdf = ee.data.computeFeatures({
        'expression': data,
        'fileFormat': 'GEOPANDAS_GEODATAFRAME'
    })
    gdf.crs = 'EPSG:4326'

    # Convert the geometry column to WKT
    gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.wkt)

    # Determine if it's the first iteration (i == 0)
    write_header = i == 0

    # overwrites if i==0 else appends
    mode = 'w' if i==0 else 'a'

    # Write the GeoDataFrame to CSV
    gdf.to_csv(csv_file_path, mode= mode, header=write_header, index=False)

## Extract HS data for 2023

# Train models

In [22]:
header = ['geometry', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'class']

gdf18 = pd.read_csv(r"C:\Users\coach\myfiles\postdoc\Invasives\data\LUCAS\Wessels_L8_30m_2018.csv", header = None, names=header)
gdf18['geometry'] = gdf18['geometry'].apply(wkt.loads)
gdf18 = gpd.GeoDataFrame(gdf18, geometry='geometry', crs= 'EPSG:4326')

In [23]:
from sklearn.ensemble import RandomForestClassifier 

wavelength18_cols = [ 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
clf18 = RandomForestClassifier(random_state=42, n_jobs=-1)
clf18.fit(gdf18[wavelength18_cols], gdf18['class'].astype(int))

RandomForestClassifier(n_jobs=-1, random_state=42)

### Evaluate 2018 map

In [ ]:
# load 2018 lucas points
# extract 2018 Landsat signatures
# predict 2018 classes
# evaluate 2018 map wth lucas classes

In [32]:
test_pts.size()

In [6]:
test_pts = ee.FeatureCollection("projects/ee-gsingh/assets/postdoc/lucas_2015_2018_change")#33734 points
train_pts = ee.FeatureCollection('projects/ee-gsingh/assets/postdoc/wessels_stratified_10ksample_2018')

# Buffer radius in meters
bufferDist = 100  

# Step 1: Buffer and merge all exclude points into a single geometry
excludeBufferUnion = (
    train_pts
    .map(lambda f: f.buffer(bufferDist, 10))
    .union(1)  # merge into one feature
    .geometry()
)

# Step 2: Filter pointsKeep to those NOT intersecting the buffer union
pointsFiltered = test_pts.filter(ee.Filter.bounds(excludeBufferUnion))
# eeprint(pointsFiltered.size()) results in 33723 points


In [8]:
pointsFiltered.aggregate_array('point_id')

In [9]:
# point_id values to exclude
exclude_ids = [
    31782030, 39362452, 46024420, 50144424,
    33721790, 40322246, 56262774, 57582514,
    43203380, 29242286, 31922314
]

# Filter out features with these point_id values
filtered = test_pts.filter(
    ee.Filter.inList('point_id', ee.List(exclude_ids)).Not()
)

# Print counts
# print('Original count:', fc.size().getInfo())
print('Filtered count:', filtered.size().getInfo())

Filtered count: 33723


In [ ]:
from tqdm.auto import tqdm
import os
grid = aoi.coveringGrid(ee.Projection('EPSG:4326').scale(2.5, 1.5))
gridList = grid.toList(972)
csv_file_path = r"C:\Users\coach\myfiles\postdoc\Invasives\data\LUCAS\Wessels_eval_L8_30m_2018.csv"

# Remove existing file to ensure clean start
if os.path.exists(csv_file_path):
    os.remove(csv_file_path)

for i in tqdm(range(0, 972)):
    fts = filtered.filterBounds(ee.Feature(gridList.get(i)).geometry())

    if fts.size().getInfo() == 0:
        print(f"No features found in grid cell {i}. Skipping...")
        continue

    composite2018 = createLandsatComposite(imageCollection= ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filter(ee.Filter.lte('CLOUD_COVER', 70)),
                points=fts.geometry(),
                year=2018)
    data = composite2018.sampleRegions(collection = fts, scale = 30, geometries = True)

    # Conversion to GeoDataFrame
    gdf = ee.data.computeFeatures({
        'expression': data,
        'fileFormat': 'GEOPANDAS_GEODATAFRAME'
    })
    gdf.crs = 'EPSG:4326'

    # Convert the geometry column to WKT
    gdf['geometry'] = gdf['geometry'].apply(lambda geom: geom.wkt)

    # Determine if it's the first iteration (i == 0)
    write_header = i == 0

    # overwrites if i==0 else appends
    mode = 'w' if i==0 else 'a'

    # Write the GeoDataFrame to CSV
    gdf.to_csv(csv_file_path, mode= mode, header=write_header, index=False)

In [18]:
gdf.columns

Index(['geometry', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7',
       'change', 'lc1', 'lc1_label', 'point_id', 'reclass15', 'reclass18',
       'year'],
      dtype='object')

In [50]:
header = ['geometry', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7',
       'change', 'lc1', 'lc1_label', 'point_id', 'reclass15', 'reclass18',
       'year']
rfeval = pd.read_csv(r"C:\Users\coach\myfiles\postdoc\Invasives\data\LUCAS\Wessels_eval_L8_30m_2018.csv", header = None, names=header)
rfeval.shape

(33723, 14)

In [51]:
from sklearn.preprocessing import LabelEncoder
rfeval.rename(columns={'reclass18': 'class'},inplace=True)

encoder = LabelEncoder()
rfeval["class_enc"] = encoder.fit_transform(rfeval["class"])

print(dict(zip(encoder.classes_, encoder.transform(encoder.classes_))))

rfeval['classification'] = clf18.predict(rfeval[['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']])

{'Artificial land': np.int64(0), 'Bare land': np.int64(1), 'Cropland': np.int64(2), 'Grassland': np.int64(3), 'Shrubland': np.int64(4), 'Water': np.int64(5), 'Wetland': np.int64(6), 'Woodland': np.int64(7)}


In [53]:
from sklearn.metrics import accuracy_score, classification_report, f1_score

y_test = rfeval['class_enc'].astype(int)  # True labels
y_pred = rfeval['classification'].astype(int)  # Predicted labels

# Accuracy
acc = accuracy_score(y_test, y_pred)

# F1
f1 = f1_score(y_test, y_pred, average='weighted')

# Report
report = classification_report(y_test, y_pred, output_dict=True)

print(f"\n Accuracy: {acc:.2f}, F1 Score: {f1:.2f}")
print(classification_report(y_test, y_pred))


 Accuracy: 0.65, F1 Score: 0.68
              precision    recall  f1-score   support

           0       0.36      0.76      0.48       782
           1       0.41      0.56      0.47       621
           2       0.75      0.58      0.65      8108
           3       0.72      0.62      0.67      8878
           4       0.32      0.42      0.36      1898
           5       0.40      0.87      0.55       309
           6       0.11      0.58      0.19       535
           7       0.83      0.76      0.79     12592

    accuracy                           0.65     33723
   macro avg       0.49      0.64      0.52     33723
weighted avg       0.72      0.65      0.68     33723

